In [1]:
import tensorflow as tf

In [2]:
print(tf.config.list_physical_devices('GPU'))

[]


In [3]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

In [4]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [5]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.61700565,  0.2815616 ,  0.09467947, -0.6033537 , -0.1821346 ,
         -0.03948337,  0.31215495, -0.21078438, -0.04917353,  0.462745  ],
        [-0.3667486 , -0.19503024,  0.577275  , -0.07165211, -0.00421077,
         -0.22796983, -0.18680996,  0.05294299, -0.5324395 , -0.22154906],
        [-0.26719686,  0.02694166,  0.5362025 ,  0.02890551,  0.1953665 ,
          0.03432822, -0.24502936, -0.17337856, -0.21357971, -0.23734587],
        [ 0.12599939,  0.50936586,  0.1650672 ,  0.43016666, -0.46891233,
          0.3216827 , -0.07070476,  0.25215656, -0.01644838, -0.50041837],
        [ 0.32335675, -0.39672098, -0.07158595, -0.03627414,  0.09031516,
          0.41907948,  0.44121867, -0.02571362,  0.01279956,  0.5690426 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [6]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.61700565,  0.2815616 ,  0.09467947, -0.6033537 , -0.1821346 ,
         -0.03948337,  0.31215495, -0.21078438, -0.04917353,  0.462745  ],
        [-0.3667486 , -0.19503024,  0.577275  , -0.07165211, -0.00421077,
         -0.22796983, -0.18680996,  0.05294299, -0.5324395 , -0.22154906],
        [-0.26719686,  0.02694166,  0.5362025 ,  0.02890551,  0.1953665 ,
          0.03432822, -0.24502936, -0.17337856, -0.21357971, -0.23734587],
        [ 0.12599939,  0.50936586,  0.1650672 ,  0.43016666, -0.46891233,
          0.3216827 , -0.07070476,  0.25215656, -0.01644838, -0.50041837],
        [ 0.32335675, -0.39672098, -0.07158595, -0.03627414,  0.09031516,
          0.41907948,  0.44121867, -0.02571362,  0.01279956,  0.5690426 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

In [7]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [8]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

In [9]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


In [10]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [11]:
_ = block(tf.zeros([1, 2, 3, 3]))

In [12]:
block.layers

In [13]:
len(block.variables)

18

In [14]:
block.summary()

Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  4         
                                                                 
 batch_normalization (Batch  multiple                  4         
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           multiple                  4         
                                                                 
 batch_normalization_1 (Bat  multiple                  8         
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           multiple                  9         
                                                                 
 batch_normalization_2 (Bat  multiple                  12        


In [15]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [16]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, None, None, 1)     4         
                                                                 
 batch_normalization_3 (Bat  (None, None, None, 1)     4         
 chNormalization)                                                
                                                                 
 conv2d_4 (Conv2D)           (None, None, None, 2)     4         
                                                                 
 batch_normalization_4 (Bat  (None, None, None, 2)     8         
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           (None, None, None, 3)     9         
                                                                 
 batch_normalization_5 (Bat  (None, None, None, 3)     1